# N2C2 Contextualized Medication Event Extraction

## Import Data

In [114]:
path = '/scratch/js12684/final/data'

In [115]:
#retrieve paths for text and ann data

import glob
notes_orig =glob.glob(path + '/*txt')
ann_orig = glob.glob(path + '/*ann' )

print(len(notes_orig))
print(len(ann_orig))

400
400


In [116]:
#create df with notes and ann paths
import pandas as pd

notes_dict = {'Notes Orig': notes_orig}
notes_df = pd.DataFrame(notes_orig).rename(columns={0: 'Notes Orig'})
notes_df.sort_values(by = 'Notes Orig', inplace = True)
notes_df.reset_index(drop = True, inplace = True)

ann_dict = {'Ann Orig': ann_orig}
ann_df = pd.DataFrame(ann_orig).rename(columns={0: 'Ann Orig'})
ann_df.sort_values(by = 'Ann Orig', inplace = True)
ann_df.reset_index(drop = True, inplace = True)

orig_df = pd.concat([notes_df, ann_df], axis = 1)
orig_df.head()

,Notes Orig,Ann Orig
0,/scratch/js12684/final/data/100-01.txt,/scratch/js12684/final/data/100-01.ann
1,/scratch/js12684/final/data/100-02.txt,/scratch/js12684/final/data/100-02.ann
2,/scratch/js12684/final/data/100-03.txt,/scratch/js12684/final/data/100-03.ann
3,/scratch/js12684/final/data/100-04.txt,/scratch/js12684/final/data/100-04.ann
4,/scratch/js12684/final/data/100-05.txt,/scratch/js12684/final/data/100-05.ann


In [117]:
#add patient id and visit num columns

patient_id = []
visit_num = []
for i in range(len(orig_df)):
  dir = orig_df['Notes Orig']
  patient_id.append(dir.iloc[i][-10:-7])
  visit_num.append(dir.iloc[i][-6:-4])

orig_df['Patient ID'] = patient_id
orig_df['Visit Num'] = visit_num

orig_df = orig_df.sort_values(by =['Patient ID', 'Visit Num'])
orig_df.head()

,Notes Orig,Ann Orig,Patient ID,Visit Num
0,/scratch/js12684/final/data/100-01.txt,/scratch/js12684/final/data/100-01.ann,100,01
1,/scratch/js12684/final/data/100-02.txt,/scratch/js12684/final/data/100-02.ann,100,02
2,/scratch/js12684/final/data/100-03.txt,/scratch/js12684/final/data/100-03.ann,100,03
3,/scratch/js12684/final/data/100-04.txt,/scratch/js12684/final/data/100-04.ann,100,04
4,/scratch/js12684/final/data/100-05.txt,/scratch/js12684/final/data/100-05.ann,100,05


In [118]:
# add full note text to new df column

notes = []
for i in range(len(orig_df)):
    f = open(orig_df['Notes Orig'].iloc[i], 'r')
    notes.append(f.read())

orig_df['Notes'] = notes

In [119]:
# add full ann text to new df column

ann = []
for i in range(len(orig_df)):
    f = open(orig_df['Ann Orig'].iloc[i], 'r')
    ann.append(f.read())

orig_df['Ann'] = ann

In [120]:
#70-10-20 train-val-test split

from sklearn.model_selection import train_test_split

train, test = train_test_split(orig_df, test_size = 0.2, shuffle = False) #false shuffle to maintain sorted order

train, val = train_test_split(train, train_size = 0.875, shuffle = False)

print(train.shape)
print(val.shape)
print(test.shape)

(280, 6)
(40, 6)
(80, 6)


In [121]:
#check patient overlap in split data
print(train['Patient ID'].iloc[-1])
print(val['Patient ID'].iloc[0])
print(val['Patient ID'].iloc[-1])
print(test['Patient ID'].iloc[0])

304
305
336
338


In [125]:
train.head()

,Notes Orig,Ann Orig,Patient ID,Visit Num,Notes,Ann
0,/scratch/js12684/final/data/100-01.txt,/scratch/js12684/final/data/100-01.ann,100,01,\n\n\nRecord date: 2106-02-12\n\nCampbell Orth...,T1\tNoDisposition 821 827\tProzac\nE1\tNoDispo...
1,/scratch/js12684/final/data/100-02.txt,/scratch/js12684/final/data/100-02.ann,100,02,\n\n\nRecord date: 2108-03-14\n\n ...,
2,/scratch/js12684/final/data/100-03.txt,/scratch/js12684/final/data/100-03.ann,100,03,\n\n\nRecord date: 2109-09-14\n\n \nSeptember ...,T3\tNoDisposition 4749 4757\tProvigil\nE3\tNoD...
3,/scratch/js12684/final/data/100-04.txt,/scratch/js12684/final/data/100-04.ann,100,04,\n\n\nRecord date: 2111-10-10\n\n \n\n\n\nC...,T2\tDisposition 830 837\theparin\nE2\tDisposit...
4,/scratch/js12684/final/data/100-05.txt,/scratch/js12684/final/data/100-05.ann,100,05,\n\n\nRecord date: 2111-12-14\n\n \n\n\n\nNEUR...,T1\tNoDisposition 1461 1471\tamiodarone\nE1\tN...


In [127]:
print(train['Notes'].iloc[0])




Record date: 2106-02-12

Campbell Orthopedic Associates
4 Madera Circle
Omak, GA 28172
 
Habib Valenzuela, M.D.
 
 
                                             Valdez, Harlan Jr.  
                                           845-41-54-4
                                             February 12, 2106 
Har is a 43 year old 6' 214 pound gentleman who is referred for
consultation by Dr. Harlan Oneil.  About a week ago he slipped on
the driveway at home and sustained an injury to his left ankle. 
He was seen at Tri-City Hospital and was told he had a
fracture.  He was placed in an air splint and advised to be
partial weight bearing, and he is using a cane.  He is here for
routine follow-up. 
Past medical history is notable for no ankle injuries previously. 
He has a history of diabetes and sleep apnea.  He takes Prozac,
Cardizem, Glucophage and Amaryl.  He is also followed by Dr. Harold
Nutter for an arrhythmia.  He does not smoke.  He drinks
minimally.  He is a set designer at Columbia P

In [128]:
print(train['Ann'].iloc[0])

T1	NoDisposition 821 827	Prozac
E1	NoDisposition:T1 
T2	NoDisposition 829 837	Cardizem
E2	NoDisposition:T2 
T3	NoDisposition 854 860	Amaryl
E3	NoDisposition:T3 
T4	NoDisposition 839 849	Glucophage
E4	NoDisposition:T4 


## Clinical BERT Model

In [1]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")


Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
